## Distributed training with Amazon SageMaker

In this notebook we use the SageMaker Python SDK to setup and run a distributed training job.
SageMaker makes it easy to train models across a cluster containing a large number of machines, without having to explicitly manage those resources. 

**Step 1:** Import essentials packages, start a sagemaker session and specify the bucket name you created in the pre-requsites section of this workshop.

In [1]:
import os
import time
import numpy as np
import sagemaker

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

bucket_name    = 'sagemaker-jobs'
jobs_folder    = 'jobs'
dataset_folder = 'datasets'

**Step 2:** Specify hyperparameters, instance type and number of instances to distribute training to. The `hvd_processes_per_host` corrosponds to number of GPUs per instances. 

In [2]:
hvd_instance_type = 'ml.p3.16xlarge'
hvd_instance_count = 4
hvd_processes_per_host = 8

print(f'Distributed training with a total of {hvd_processes_per_host*hvd_instance_count} workers: {hvd_instance_count} instances of {hvd_instance_type}')
print(f'{hvd_processes_per_host} GPU(s) per instance')

Distributed training with a total of 32 workers: 4 instances of ml.p3.16xlarge
8 GPU(s) per instance


In [3]:
job_name   = f'tf-horovod-{hvd_instance_count}x{hvd_processes_per_host}-workers-{time.strftime("%Y-%m-%d-%H-%M-%S-%j", time.gmtime())}'
output_path = f's3://{bucket_name}/{jobs_folder}'
tboard_logs = f's3://{bucket_name}/tensorboard_logs/{job_name}'

metric_definitions = [{'Name': 'val_acc', 'Regex': 'val_acc: ([0-9\\.]+)'}]

hyperparameters = {'epochs': 100, 
                   'learning-rate': 0.001,
                   'momentum': 0.9,
                   'weight-decay': 2e-4,
                   'optimizer': 'adam',
                   'batch-size' : 256}

sm_config       = {'tensorboard_logs': tboard_logs}

hyperparameters.update(sm_config)

**Step 3:** In this cell we create a SageMaker estimator, by providing it with all the information it needs to launch instances and execute training on those instances.

Since we're using horovod for distributed training, we specify `distributions` to mpi which is used by horovod.

In the TensorFlow estimator call, we specify training script under `entry_point` and dependencies under `code`. SageMaker automatically copies these files into a TensorFlow container behind the scenes, and are executed on the training instances.

In [4]:
distributions = {
                 'mpi': {
                          'enabled'           : True,
                          'processes_per_host': hvd_processes_per_host,
                          'custom_mpi_options': '-verbose --NCCL_DEBUG=INFO -x OMPI_MCA_btl_vader_single_copy_mechanism=none'
                        }
                }

In [5]:
from sagemaker.tensorflow import TensorFlow
hvd_estimator = TensorFlow(entry_point         = 'cifar10-tf-horovod-sagemaker.py', 
                          source_dir           = 'code',
                          output_path          = output_path + '/',
                          code_location        = output_path,
                          role                 = role,
                          train_instance_count = hvd_instance_count, 
                          train_instance_type  = hvd_instance_type,
                          framework_version    = '1.15', 
                          py_version           = 'py3',
                          script_mode          = True,
                          metric_definitions   = metric_definitions,
                          hyperparameters      = hyperparameters,
                          distributions        = distributions)

**Step 4:** Specify dataset locations in Amazon S3 and then call the fit function.

In [6]:
train_path = f's3://{bucket_name}/{dataset_folder}/cifar10-dataset/train'
val_path   = f's3://{bucket_name}/{dataset_folder}/cifar10-dataset/validation'
eval_path  = f's3://{bucket_name}/{dataset_folder}/cifar10-dataset/eval'

hvd_estimator.fit({'train': train_path,'validation': val_path,'eval': eval_path}, 
                  job_name=job_name, wait=False)

**Note**: in the `estimator_hvd.fit()` function above, change`wait=True` if you want to see the training output in the Jupyter notebook.
Advantage of setting `wait=False`, is that you can continue to run cells. 
Since we're unblocked due to `wait=False` we can now launch tensorboard in the notebook and monitor progress.

**Step 5:** Monitor progress on TensorBoard. Launch tensorboard server and open the link on a new tab to visualize training progress, and navigate to the following link

In [7]:
# !S3_REGION=us-west-2 tensorboard --logdir s3://{bucket_name}/tensorboard_logs/

Open a new browser and navigate to the folloiwng link to access TensorBoard:
<br> https://localhost:6006
